In [1]:
import pandas as pd
import numpy as np
from features.get_all_DD_features import *
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
chat_df = pd.read_csv('./data/raw_data/juries_tiny_for_testing.csv', encoding='mac-roman')
vect_df = pd.read_csv('./data/vectors/sentence/chats/juries_tiny_for_testing.csv')

In [3]:
chat_df['message_embedding'] = conv_to_float_arr(vect_df['message_embedding'].to_frame())
chat_df

,batch_num,round_num,speaker_hash,speaker_nickname,timestamp,message,majority_pct,num_flipped,flipped_pct,num_votes,message_embedding
0,0,0,5e7e1e0031f4e454e196c30b,niceRhino,2020-04-20T18:27:20.125Z,Hello!,1.0,1,0.333333,3,"[-0.06062672659754753, 0.03996486961841583, 0...."
1,0,0,5e31d6e4e31c5304c46f1413,culturedCow,2020-04-20T18:27:23.764Z,Hi!,1.0,1,0.333333,3,"[-0.08946915715932846, 0.03267911076545715, 0...."
2,0,0,5e7e4f4c31f4e454e196c9c4,spryBison,2020-04-20T18:27:27.724Z,Hello,1.0,1,0.333333,3,"[-0.0627717524766922, 0.05495874583721161, 0.0..."
3,0,0,5d482ea421c9be351f762255,youngLion,2020-04-20T18:27:30.410Z,Hi,1.0,1,0.333333,3,"[-0.09047631174325943, 0.04043954238295555, 0...."
4,0,0,5e84cc3c50f6e364321d6265,smallGiraffe,2020-04-20T18:27:35.506Z,hi,1.0,1,0.333333,3,"[-0.09047631174325943, 0.04043954238295555, 0...."
...,...,...,...,...,...,...,...,...,...,...,...
92,0,2,5e7e4f4c31f4e454e196c9c4,newLion,2020-04-20T19:02:55.111Z,I say asshole under stress,0.6,0,0.000000,5,"[0.03895288333296776, -0.03848191350698471, -0..."
93,0,2,5d6feec65f80ae21f5c5f054,conventionalMonkey,2020-04-20T19:03:21.819Z,"Yes, she is the asshole... unfortunately. Husb...",0.6,0,0.000000,5,"[0.038630757480859756, 0.04088406637310982, 0...."
94,0,2,5d482ea421c9be351f762255,newPanda,2020-04-20T19:03:36.308Z,I think she is being presumptuous and acting l...,0.6,0,0.000000,5,"[-0.014676126651465893, -0.06470786780118942, ..."
95,0,2,5e7e4f4c31f4e454e196c9c4,newLion,2020-04-20T19:03:53.219Z,"Tha's true, she ins't considering her husband ...",0.6,0,0.000000,5,"[-0.01419044565409422, -0.0679955780506134, 0...."


In [4]:
# METHOD 1: calculate avg cosine similarity between adjacent chats

lex_cohesion = []
fflow = []

for num, conv in chat_df.groupby(['batch_num', 'round_num']):
    
    for i, pair in conv.groupby(conv.index // 2):
        
        # last "pair" has only one element, safeguard against this
        if (len(pair) == 2):
            cos_sim_matrix = cosine_similarity([pair.iloc[0]['message_embedding'], pair.iloc[1]['message_embedding']])
            lex_cohesion.append(cos_sim_matrix[np.triu_indices(len(cos_sim_matrix), k = 1)][0])
    
    fflow.append(sum(lex_cohesion) / len(conv))


In [5]:
final = chat_df[['batch_num', 'round_num']].drop_duplicates()
final['lex_cohesion'] = fflow
final

,batch_num,round_num,lex_cohesion
0,0,0,0.166486
52,0,2,0.318432


In [9]:
# METHOD 2: calculate avg cosine similarity between chat and it's preceding chats FOR EVERY CHAT

lex_cohesion = []
fflow = []
cached_cohesion = 0
pair_index = 1

for num, conv in chat_df.groupby(['batch_num', 'round_num']):

    for i, pair in conv.groupby(conv.index // 2):
        
        # last "pair" has only one element, safeguard against this
        if (len(pair) == 2):
            
            cos_sim_matrix = cosine_similarity([pair.iloc[0]['message_embedding'], pair.iloc[1]['message_embedding']])
            cached_cohesion += cos_sim_matrix[np.triu_indices(len(cos_sim_matrix), k = 1)][0]
            
            lex_cohesion.append(cached_cohesion/pair_index)
    
    fflow.append(sum(lex_cohesion) / len(conv))


In [8]:
final = chat_df[['batch_num', 'round_num']].drop_duplicates()
final['lex_cohesion'] = fflow
final

,batch_num,round_num,lex_cohesion
0,0,0,2.603752
52,0,2,8.541680
